In [1]:
!git clone https://github.com/marketneutral/kuma_utils.git
%cd kuma_utils
!git checkout lgb_add_tqdm
%cd ..
import sys
sys.path.append("kuma_utils/")

Cloning into 'kuma_utils'...
remote: Enumerating objects: 894, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 894 (delta 93), reused 90 (delta 90), pack-reused 794
Receiving objects: 100% (894/894), 668.74 KiB | 2.67 MiB/s, done.
Resolving deltas: 100% (581/581), done.
/kaggle/working/kuma_utils
Branch 'lgb_add_tqdm' set up to track remote branch 'lgb_add_tqdm' from 'origin'.
Switched to a new branch 'lgb_add_tqdm'
/kaggle/working


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from kuma_utils.preprocessing.imputer import LGBMImputer

In [3]:
input_path = Path('/kaggle/input/tabular-playground-series-jun-2022/')
data = pd.read_csv(input_path / 'data.csv', index_col='row_id')
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='row-col')

In [4]:
for col in data.columns:
    if data[col].dtype == "float64":
        data[col]=pd.to_numeric(data[col], downcast="float")
    if data[col].dtype == "int64":
        data[col]=pd.to_numeric(data[col], downcast="integer")

In [5]:
F = [[], [], [], [], []]
features = list(data.columns)

for feature in features:
    for i in [1, 2, 3, 4]:
        if feature.split('_')[1] == str(i):
            F[i].append(feature)
df = [[], [], [], [], []]

for i in [1, 2, 3, 4]:
    df[i] = data[F[i]]
    corr = df[i].corr()

In [6]:
for i in [1, 3, 4]:
    imp = LGBMImputer(n_iter=100, verbose=True)
    df[i][:] = imp.fit_transform(df[i])

  0%|          | 0/15 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
d = pd.concat([df[1], df[2], df[3], df[4]], axis=1)
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = d.loc[row, col]

submission.to_csv('sub_9.csv')

100%|██████████| 1000000/1000000 [01:44<00:00, 9580.57it/s]
